In [ ]:
from pystac_client import Client
from odc.stac import load

In [ ]:
latlon_min = (-10.226241126118127, 150.9049415588338)
latlon_max = (-9.881091690881888, 151.37323379516192)

bbox = [latlon_min[1], latlon_min[0], latlon_max[1], latlon_max[0]]

In [ ]:
catalog = "https://stac.staging.digitalearthpacific.io/"
client = Client.open(catalog)

In [ ]:
items = client.search(collections="dep_s2_geomad", bbox=bbox).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(items, bbox=bbox, chunks={"x": 2048, "y": 2048})
data

In [ ]:
import rasterio

with rasterio.Env(AWS_NO_SIGN_REQUEST="YES"):
    loaded = data[["red", "green", "blue", "smad", "bcmad", "emad"]].compute()

In [ ]:
import folium

m = folium.Map(location=loaded.odc.geobox.extent.centroid.to_crs("epsg:4326").coords[0][::-1], zoom_start=10)

loaded.odc.to_rgba(vmin=1000, vmax=3000).odc.add_to(m)

for band in ["smad", "emad", "bcmad"]:
    loaded[band].odc.add_to(m, robust=True)

folium.LayerControl().add_to(m)

m

In [ ]:
# m.save("index.html")